In [86]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.base import TransformerMixin
from sklearn_pandas import DataFrameMapper
import seaborn as sns
import matplotlib
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import Imputer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [87]:
train = pd.read_csv('train.csv')#.drop('PassengerId', axis=1)
test = pd.read_csv('test.csv')#.drop('PassengerId', axis=1)
alldata = pd.concat([train.drop('Survived', axis=1),test])

# Exploring Data

In [88]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [89]:
alldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB


**PassengerId** - irrelevant in terms of features (already droped)<br>
**Survived** - "1" if survived, "0" otherwise <br>
**Pclass** - one of three classes (1,2,3) <br>
**Name** - name of a passenger including their titles<br>
**Sex** - male or female<br>
**Age** - age in years <br>
**SibSp** - number of passenger's siblings and spauses travelling with them<br>
**Parch** - number of passenger's parents and children travelling with them<br>
**Ticket** - ticket number<br>
**Fare** - fare<br>
**Cabin** - letters encode deck number and numbers encode cabin number <br>

**Missing Data:** <br>
'Age': 263 missing samples<br>
'Cabin': 1014 missing samples (almost 80% of data missing!)<br>
'Emarked': only 2 missing samples

**Intuition**<br>
Sex, Pclass, Fare and Age may play an important role in classification.<br>
Cabin, and even more the deck encoded in this feature may be a very important factor. There are planty of samples missing though.<br>
Noble title which can be extracted from Name can be a good feature.


In [90]:
train.corr(method='spearman')

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.034091,0.041010,-0.061161,0.001235,-0.013975
Survived,-0.005007,1.000000,-0.339668,-0.052565,0.088879,0.138266,0.323736
Pclass,-0.034091,-0.339668,1.000000,-0.361666,-0.043019,-0.022801,-0.688032
Age,0.041010,-0.052565,-0.361666,1.000000,-0.182061,-0.254212,0.135051
SibSp,-0.061161,0.088879,-0.043019,-0.182061,1.000000,0.450014,0.447113
Parch,0.001235,0.138266,-0.022801,-0.254212,0.450014,1.000000,0.410074
Fare,-0.013975,0.323736,-0.688032,0.135051,0.447113,0.410074,1.000000


## Title Encoding 

Name is determied for each sample. We can use it to extract all the noble titles  

In [91]:
set([i.split(', ')[1].split('.')[0] for i in alldata['Name']])

{'Capt',
 'Col',
 'Don',
 'Dona',
 'Dr',
 'Jonkheer',
 'Lady',
 'Major',
 'Master',
 'Miss',
 'Mlle',
 'Mme',
 'Mr',
 'Mrs',
 'Ms',
 'Rev',
 'Sir',
 'the Countess'}

In [92]:
class TitleEncoder(TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        transformed = X.copy()
        for i in range(len(X)):
            transformed[i] = X[i].split(',')[1].split('.')[0]
        return transformed

class DeckEncoder(TransformerMixin):
    def fit(self,X, y=None):
        return self
    def transform(self, X):
        transformed = X.copy()
        for i in range(len(X)):
            try:
                transformed[i] = X[i][0]
            except TypeError:
                transformed[i] = ' '
        return transformed
    
class SeriesImputer(Imputer):
    def fit(self, X, y=None):
        return super(SeriesImputer, self).fit(np.reshape(X.values,(-1, 1)), y=y)

    def transform(self, X):
        return super(SeriesImputer, self).transform(np.reshape(X.values,(-1, 1)))

## Ticket Encoding

In [93]:
mapper = DataFrameMapper([
    #features
    ('Pclass', sklearn.preprocessing.LabelBinarizer()),
    ('Sex', sklearn.preprocessing.LabelBinarizer()),
    ('Name', [TitleEncoder(),sklearn.preprocessing.LabelBinarizer()],{'alias': 'title'}),
    ('Fare', SeriesImputer()),
    ('Age', SeriesImputer()),
    ('SibSp', None),
    ('Parch', None),
    #result
    #('Survived', None)
], input_df=True, df_out=True)

In [94]:
data = mapper.fit_transform(train, train.Survived)

In [95]:
X = np.array(data.values, dtype='float')
y = np.array(train['Survived'].values, dtype='float')

# Random Forest

In [96]:
param_grid = { 
    'n_estimators': [30, 50,100,200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [2,3,4,5,6,7,8,10],
    'criterion' :['gini', 'entropy']
}
clf_forest = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=clf_forest, param_grid=param_grid, cv= 5)
grid_search.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [30, 50, 100, 200, 500], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [2, 3, 4, 5, 6, 7, 8, 10], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [97]:
print(grid_search.best_params_)
clf_forest = RandomForestClassifier(**grid_search.best_params_)
scores = cross_val_score(clf_forest, X, y, cv=10)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

{'criterion': 'gini', 'max_depth': 10, 'max_features': 'log2', 'n_estimators': 200}
[0.75555556 0.84444444 0.74157303 0.88764045 0.93258427 0.86516854
 0.83146067 0.79775281 0.86516854 0.85227273]
Accuracy: 0.84 (+/- 0.11)


# Submit

In [98]:
test_data = mapper.transform(test)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 25 columns):
Pclass_1               418 non-null int64
Pclass_2               418 non-null int64
Pclass_3               418 non-null int64
Sex                    418 non-null int64
title_ Capt            418 non-null int64
title_ Col             418 non-null int64
title_ Don             418 non-null int64
title_ Dr              418 non-null int64
title_ Jonkheer        418 non-null int64
title_ Lady            418 non-null int64
title_ Major           418 non-null int64
title_ Master          418 non-null int64
title_ Miss            418 non-null int64
title_ Mlle            418 non-null int64
title_ Mme             418 non-null int64
title_ Mr              418 non-null int64
title_ Mrs             418 non-null int64
title_ Ms              418 non-null int64
title_ Rev             418 non-null int64
title_ Sir             418 non-null int64
title_ the Countess    418 non-null int64
Fare         

In [99]:
X_test = np.array(test_data.values, dtype='float')

In [100]:
clf_forest.fit(X,y)
predictions = clf_forest.predict(X_test).astype(int)

In [101]:
submission = pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':predictions})

In [102]:
filename = 'Titanic Predictions 1.csv'

submission.to_csv(filename,index=False)
